# 6.1 パラメータチューニング

## 6.1.1 ハイパーパラメータの探索手法

1. 手動
2. グリッドサーチ/ランダムサーチ  
   パラメータ空間を予め指定した範囲を規則的に探索/予め指定した手法でランダムに探索  
   `sckikit-learn.model_selection` の `GridSearchCV`, `RandomizedSearchCV` など
3. ベイズ最適化 (Bayseian Optimization)  
    以前に計算したパラメータの履歴に基づいてベイズの手法を用いて選択  
    `hyperopt`, `optuna` など

kaggle でのパラメータ探索は手で行っている人が多い印象。

## 6.1.2 パラメータチューニングで設定すること

1. ベースラインとなるパラメータ
2. 探索する対象となるパラメータとその範囲
3. 手動で調節するか、自動的に探索するか
4. 評価の枠組み (fold の分け方など)

パラメータを自動で調節する場合には、

- パラメータチューニングをしすぎて学習データに過剰に適合してしまう
- 計算時間が長くなる

といった問題が起こり得るので、

- チューニングとモデルの作成をする際の fold の分け方を変える
- validation の fold の 1 つだけを用いて精度を確認する

などの対策をしたほうが良い。

## 6.1.3 パラメータチューニングのポイント

各モデルには結果を大きく左右する大事なパラメータが存在する。  
そのため、パラメータを探索する際はその**モデルの重要なパラメータから調節していく**ことが大事である。  

また、**モデルとパラメータの関係を理解して得られた結果からなぜそうなったのかを理解すること**で、次にどうパラメータを変化させるべきかを考えながらチューニングしていくとよい。

なお、GBDT ではパラメータチューニングよりも良い特徴量を追加するほうが有用なことが多いので、あまりチューニングに時間を割かない方が良い。


## 6.1.4 ベイズ最適化でのパラメータ探索

Tree-structured Parzen Estimator (TPE) というアルゴリズムを用いて最適化を行っている `hyperopt` と `optuna` について、以下具体的な使い方を見ていく。

In [ ]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]  # 最初の fold のみ用いる
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# xgboostによる学習・予測を行うクラス
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'eval_metric': 'error', 'verbosity': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, verbose_eval=False, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

### [`hyperopt`](http://hyperopt.github.io/hyperopt/)

具体的な手順は以下のよう。

1. 最小化したい評価指標を返す関数を作成する (`score` function) 
2. 探索するパラメータ範囲を定義する (`space` 変数)
3. 探索回数を指定する (`max_eval` 変数)

以上を `hyperopt.fmin` 関数に代入して探索を行う。

経験的には 25 回程度の探索でそれなりに妥当なパラメータが見つかり始め、100 回程度で十分な探索が行われるようである。


In [ ]:
# -----------------------------------
# hyperopt を使ったパラメータ探索
# -----------------------------------
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# 探索するパラメータの空間を指定する
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限・間隔
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

# hyperoptによるパラメータ探索の実行
max_evals = 20
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

### [`optuna`](https://optuna.readthedocs.io/en/stable/)

2018 年末に公開されたフレームワークで、最適化のアルゴリズムに TPE を用いているのは hyperopt と同じだが、

- 学習曲線を用いた試行の枝切り
- 並列分散最適化

といった点でより効率的になっている。

以下で見るように確かに簡単に使えるし、個人的に最近よく聞くのはこっちな気がする。

In [ ]:
# -----------------------------------
# optuna を使ったパラメータ探索
# -----------------------------------
import optuna

def objective(trial):

    params = {
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'gamma': trial.suggest_uniform('gamma', 0, 0.4),
    }
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)

    print(f'params: {params}, logloss: {score:.4f}')

    return score

study = optuna.create_study()
study.optimize(objective, n_trials=20)

print(f'best params:{study.best_params}, score:{study.best_value:.4f}')

ただ、ベイズ最適化によるチューニングでは以下のような問題が生じることがある。

- 計算時間のかかりすぎる試行  
  学習率を小さくした場合は学習がなかなか進まなくなってしまうので、事前に調整しておくなどの対策が必要。
- パラメータ間の依存性
  チューニングされるパラメータ同士は完全に独立でないため、依存性が強く表れる場合は効率的な探索ができない可能性がある。  
  パラメータ空間を明示的にするか、試行回数を増やす必要がある。
- 評価のランダム性によるばらつき
  評価のぶれが大きいときは効果的に探索できないので、cross validation の平均で評価するなど試行回数を増やすなどの必要がある。

従って、ベイズ最適化を使う場合はこれらの点に注意すること。

## 6.1.5 GBDT のパラメータおよびそのチューニング

ここでは、GBDT の例として xgboost のパラメータを見ていく。  
なお、lightgbm も考え方はほぼ同じである。

| parameter          | explanation                                                      | 
| :----------------: | :--------------------------------------------------------------: | 
| `eta`              | 学習率 (予測値のアップデートの際に予測誤差に乗じられる)              | 
| `num_round`        | 決定木の本数                                                     | 
| `max_depth`        | 決定木の深さ                                                     | 
| `min_child_weight` | 葉を分岐するために必要な最低限のデータ数                         | 
| `gamma`            | 決定木を分岐させるために最低限減らさなくてはいけない目的関数の値 | 
| `colsample_bytree` | 決定木ごとに特徴量の列をサンプリングする割合                     | 
| `subsmaple`        | 決定機ごとに学習データの行をサンプリングする割合                 | 
| `alpha`            | 決定木の葉の weight に対する L1 正則化の強さ                      | 
| `lambda`           | 決定木の葉の weight に対する L2 正則化の強さ                      | 

まずは、学習率などといった最重要のパラメータを以下の方針で決定する。

- `eta` を小さくしすぎると学習に時間がかかるようになるので、初めのうちは 0.1 程度にしより精度を求める段階になれば 0.01 - 0.05 程度にする 
- `num_round` は 1000 程度と大きくしておいて、`early_stopping_rounds` で制御する
- `early_stopping_rounds` は 50 程度で十分

次に、以下の学習の複雑さに関するパラメータを順次調節する。  
経験的には `max_depth` が最も重要であるらしい。

- `max_depth`, `min_child_weight`, `gamma`  
- `alpha`, `lambda`
- `subsample`, `colsample_bytree`

具体的なチューニングの例は p.318 - p.320 の COLUMN を参考に。(結構大事な気がしている)

## 6.1.6 ニューラルネットのパラメータおよびそのチューニング

調節対象となり得るパラメータ

- ネットワークの構成
  - 中間層の活性化関数 (基本は ReLU $f(x) = \max (0, x)$)
  - 中間層の層数
  - 各層のユニット数、ドロップアウト率
  - Batch normalization 層をどうするか  
  Batch normalization とは、ネットワーク内でデータの分布が偏ってしまうことを防ぐためにデータを標準化する手法。  
  大きな学習係数が使える・正則化効果があるなどのメリットがあるらしい
- オプティマイザの選択  
SGD と Adam などを試してみるとよい
- その他
  - バッチサイズ
  - weight decay (cost function の L2 regularization) などの正則化の導入やオプティマイザの学習率以外の調整

Neural net の場合、目的関数の発散を防ぐためまずは学習率を調整し、次にその他のパラメータを調節するとよい。  

Neural net のパラメータの詳しい設定・チューニングについては 4.4.6 を、
具体的なチューニングの例は p.321 - p.326 の COLUMN を参考に。

## 6.1.7 線形モデルのパラメータおよびそのチューニング

線形モデルでは正則化のパラメータがチューニング対象となる[<sup>1</sup>](#fn1)。  
対象のパラメータが少なく計算も比較的早いので、広範囲を探索することが可能である。


<span id="fn1"> [正則化（Ridge,Lasso）](https://qiita.com/greatonbi/items/0323d420af46d3ed9183) </span>